In [7]:
'''
Optimize the query plan

Suppose we want to compose query in which we get for each question also the number of answers to this question for each month. See the query below which does that in a suboptimal way and try to rewrite it to achieve a more optimal plan.
'''

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, month

import os

spark = SparkSession.builder.appName('Optimize I').getOrCreate()

base_path = os.getcwd()
print(f'base_path is: {base_path}')

project_path = ('/').join(base_path.split('/')[0:-3]) 

#answers_input_path = os.path.join(project_path, 'data/answers')
answers_input_path = os.path.join(base_path, 'data/answers')

#questions_input_path = os.path.join(project_path, 'output/questions-transformed')
questions_input_path = os.path.join(base_path, 'data/questions')

answersDF = spark.read.option('path', answers_input_path).load()

questionsDF = spark.read.option('path', questions_input_path).load()

base_path is: /Users/nancypham/springboard/spark-optimization


In [8]:
answersDF.show()

+-----------+---------+--------------------+--------+-------+-----+
|question_id|answer_id|       creation_date|comments|user_id|score|
+-----------+---------+--------------------+--------+-------+-----+
|     226592|   226595|2015-12-29 15:46:...|       3|  82798|    2|
|     388057|   388062|2018-02-22 10:52:...|       8|    520|   21|
|     293286|   293305|2016-11-17 13:35:...|       0|  47472|    2|
|     442499|   442503|2018-11-21 22:34:...|       0| 137289|    0|
|     293009|   293031|2016-11-16 05:36:...|       0|  83721|    0|
|     395532|   395537|2018-03-24 22:51:...|       0|   1325|    0|
|     329826|   329843|2017-04-29 08:42:...|       4|    520|    1|
|     294710|   295061|2016-11-26 17:29:...|       2| 114696|    2|
|     291910|   291917|2016-11-10 02:56:...|       0| 114696|    2|
|     372382|   372394|2017-12-03 18:17:...|       0| 172328|    0|
|     178387|   178394|2015-04-25 10:31:...|       6|  62726|    0|
|     393947|   393948|2018-03-17 15:22:...|    

In [9]:
questionsDF.show()

+-----------+--------------------+--------------------+--------------------+------------------+--------+-------+-----+
|question_id|                tags|       creation_date|               title|accepted_answer_id|comments|user_id|views|
+-----------+--------------------+--------------------+--------------------+------------------+--------+-------+-----+
|     382738|[optics, waves, f...|2018-01-27 23:22:...|What is the pseud...|            382772|       0|  76347|   32|
|     370717|[field-theory, de...|2017-11-25 01:09:...|What is the defin...|              null|       1|  75085|   82|
|     339944|[general-relativi...|2017-06-17 13:32:...|Could gravitation...|              null|      13| 116137|  333|
|     233852|[homework-and-exe...|2016-02-04 13:19:...|When does travell...|              null|       9|  95831|  185|
|     294165|[quantum-mechanic...|2016-11-22 03:39:...|Time-dependent qu...|              null|       1| 118807|   56|
|     173819|[homework-and-exe...|2015-04-02 08:

In [10]:
'''
Answers aggregation

Here we : get number of answers per question per month
'''
import time
start_time = time.time()
# Original version
answers_month = answersDF.withColumn('month', month('creation_date')).groupBy('question_id', 'month').agg(count('*').alias('cnt'))

resultDF = questionsDF.join(answers_month, 'question_id').select('question_id', 'creation_date', 'title', 'month', 'cnt')

resultDF.orderBy('question_id', 'month').show()

print("-- %s seconds to process --" % (time.time() - start_time))

+-----------+--------------------+--------------------+-----+---+
|question_id|       creation_date|               title|month|cnt|
+-----------+--------------------+--------------------+-----+---+
|     155989|2014-12-31 17:59:...|Frost bubble form...|    2|  1|
|     155989|2014-12-31 17:59:...|Frost bubble form...|   12|  1|
|     155990|2014-12-31 18:51:...|The abstract spac...|    1|  1|
|     155990|2014-12-31 18:51:...|The abstract spac...|   12|  1|
|     155992|2014-12-31 19:44:...|centrifugal force...|   12|  1|
|     155993|2014-12-31 19:56:...|How can I estimat...|    1|  1|
|     155995|2014-12-31 21:16:...|Why should a solu...|    1|  3|
|     155996|2014-12-31 22:06:...|Why do we assume ...|    1|  2|
|     155996|2014-12-31 22:06:...|Why do we assume ...|    2|  1|
|     155996|2014-12-31 22:06:...|Why do we assume ...|   11|  1|
|     155997|2014-12-31 22:26:...|Why do square sha...|    1|  3|
|     155999|2014-12-31 23:01:...|Diagonalizability...|    1|  1|
|     1560

In [11]:
resultDF.explain(True)

== Parsed Logical Plan ==
'Project ['question_id, 'creation_date, 'title, 'month, 'cnt]
+- Project [question_id#250L, tags#251, creation_date#252, title#253, accepted_answer_id#254L, comments#255L, user_id#256L, views#257L, month#332, cnt#348L]
   +- Join Inner, (question_id#250L = question_id#238L)
      :- Relation [question_id#250L,tags#251,creation_date#252,title#253,accepted_answer_id#254L,comments#255L,user_id#256L,views#257L] parquet
      +- Aggregate [question_id#238L, month#332], [question_id#238L, month#332, count(1) AS cnt#348L]
         +- Project [question_id#238L, answer_id#239L, creation_date#240, comments#241L, user_id#242L, score#243L, month(cast(creation_date#240 as date)) AS month#332]
            +- Relation [question_id#238L,answer_id#239L,creation_date#240,comments#241L,user_id#242L,score#243L] parquet

== Analyzed Logical Plan ==
question_id: bigint, creation_date: timestamp, title: string, month: int, cnt: bigint
Project [question_id#250L, creation_date#252, ti

In [12]:
# Optimized version 1 -- switch join order
start_time = time.time()

answers_month = answersDF.withColumn('month', month('creation_date')).groupBy('question_id', 'month').agg(count('*').alias('cnt'))

resultDF = answers_month.join(questionsDF, 'question_id').select('question_id', 'creation_date', 'title', 'month', 'cnt')

resultDF.orderBy('question_id', 'month').show()

print("-- %s seconds to process --" % (time.time() - start_time))

+-----------+--------------------+--------------------+-----+---+
|question_id|       creation_date|               title|month|cnt|
+-----------+--------------------+--------------------+-----+---+
|     155989|2014-12-31 17:59:...|Frost bubble form...|    2|  1|
|     155989|2014-12-31 17:59:...|Frost bubble form...|   12|  1|
|     155990|2014-12-31 18:51:...|The abstract spac...|    1|  1|
|     155990|2014-12-31 18:51:...|The abstract spac...|   12|  1|
|     155992|2014-12-31 19:44:...|centrifugal force...|   12|  1|
|     155993|2014-12-31 19:56:...|How can I estimat...|    1|  1|
|     155995|2014-12-31 21:16:...|Why should a solu...|    1|  3|
|     155996|2014-12-31 22:06:...|Why do we assume ...|    1|  2|
|     155996|2014-12-31 22:06:...|Why do we assume ...|    2|  1|
|     155996|2014-12-31 22:06:...|Why do we assume ...|   11|  1|
|     155997|2014-12-31 22:26:...|Why do square sha...|    1|  3|
|     155999|2014-12-31 23:01:...|Diagonalizability...|    1|  1|
|     1560